In [1]:
from multiprocessing import Process, Pool
from queue import Queue
# from functools import partial
import subprocess
import threading
import time, datetime
import glob
import smtplib
import os

os.chdir('/home/gravishlab/Documents/Python')
from ParallelCams.TriggerOneCam import *
from ParallelCams.AnalyzeOneVideo import *
from ParallelCams.PlaySound import *
from ParallelCams.OpenAirValve import *


# %matplotlib qt4
# %matplotlib auto

print('done importing')

Imported stuff


/home/gravishlab/.virtualenvs/ants/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Imported videos
done importing


In [2]:
%qtconsole

## USER INPUT: Define variables of recording program

In [3]:
# How long would you like to force the cameras to wait before recording again?
rec_interval = 60 # in seconds

# How many days of recordings?
nDays = 1

# What time would you like recording to start?
hStart = 6 # start recording at 8:00 AM

# What time would you like recording to stop?
hStop = 14 # stop at 2:00 PM - USE MILITARY TIME

# How often would you like the programs to restart (ensures that aren't internal RecordCamera errors)?
restart_time = 900 # after how many seconds should programs restart? 900 s = 15 min

# What folder will these files go into?
# folderOI = '/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180315'

# What sound parameters would you like to play?
# vol = 0.05 # between 0 and 1
# dur = 3 # in second
# fr = 300 # frequency of sin wave to play
# a_spacing = 1 # minutes between playing sound

# what open air valve parameters would you like?
a_spacing = 5 # minutes between air valve opening
n_repeat = 2 # how many times open valve in a session
spacing_sec = 1 # how many seconds between air bursts
air_start_time = 9 # what hour should the air perturbation start?
air_stop_time = 9 # what hour should the air perturbation stop?


# are webcams reversed (is webcam 1 = 12,18 and webcam 0 = 35,36)?
invert_wcams = True

## EXPERIMENT INFORMATION: What days and substrate orders were used?

In [4]:
# What cameras are you recording from?
CamNums = [12,18,35,36]

# What days did you record for each colony?
recording_days = [[13],[14],[15],[16],[17],[21],[22],[23],[24],[25]]

# What substrates are you testing?
# Subs = [0, 1, 3, 5] # in mm
Subs = ['Flat','Step','C_Step','Array']

# For each colony, what was the random ordering of the substrates?
# order goes for cams [12, 18, 35, 36]
sub_orders = [  [3, 1, 2, 4] # colony 1 - 8/13/19
              , [4, 2, 1, 3] # colony - 8/14/19
              , [1, 4, 3, 2] # colony - 8/15/19
              , [2, 3, 4, 1] # colony - 8/16/19
              , [3, 2, 1, 4] # colony - 8/17/19
              , [4, 1, 2, 3] # colony - 8/21/19
              , [1, 4, 3, 2] # colony - 8/22/19
              , [3, 2, 4, 1] # colony - 8/23/19
              , [2, 3, 4, 1] # colony - 8/24/19
              , [2, 3, 4, 1] # colony - 2/25/19
             ]


## Text message info
The code will send you text messages to alert you when sections have started and completed

In [5]:
# i created a gmail account just for this purpose
username = 'codefromglenna'
password = 'imacomputer'
fromaddr = 'codefromglenna@gmail.com'
toaddr   = '6106392662@vtext.com'

# server = smtplib.SMTP('smtp.gmail.com:587');
# based on http://github.com/CrakeNotSnowman/Python_Message/blob/master/sendMessage.py

## Define main functions:
#### 1) get user input to close down program
#### 2) read webcams and FLIR cameras
#### 3) kill all processes

In [6]:
# define thread of getting user input to close down
def get_input(CamNums):
    global flag
    print('')
    keystrk=input('\n\nPress a key to exit: ')
    # thread doesn't continue until key is pressed
    flag=False
    print('\nYou pressed: ', keystrk, ' -- PROGRAM EXITING\n')
    print('')

    return flag;



# define thread of reading/recording
def cams(CamNums, interval):
    procs = []
    
    # how many webcams need (2 point grey to every webcam)
    nWCams = int(len(CamNums)/2)+(len(CamNums) % 2 > 0) # always rounds up
    for wCamNum in range(0,nWCams):
        if invert_wcams:
            relevant_cams = np.split(np.array(CamNums), 2)[::-1][wCamNum]
        else:
            relevant_cams = np.split(np.array(CamNums), 2)[wCamNum]
        #print(CamNum)
        p1 = Process(target=startwebcam, args = (wCamNum, relevant_cams, interval))
        p1.start()
        procs.append(p1)

    for CamNum in CamNums:
        p2 = Process(target=recordcam, args = (CamNum,))
        p2.start()
        _
    while(True):
        wcams_alive = [xx.is_alive() for xx in procs]
        if not all(wcams_alive):
            return
        
        
# define how to kill all programs
def killallprocesses():
    print('Closing processes...')
    killwebcams()
    # delete trigger files
    tfiles = glob.glob('/media/gravishlab/SeagateExpansionDrive/AntTrack/*.txt');
    for tfile in tfiles:
        os.system('gvfs-trash %s' % tfile)  # move to trash
#         print('---Waiting to kill RecordCam functions')
    time.sleep(15)
    s=threading.Thread(target=killrecordcams, args = (CamNums,))
    s.start()
    s.join()
    return True;


def sendtxt(msg):
    server = smtplib.SMTP('smtp.gmail.com:587');
#     server.connect();
    server.starttls();
    server.login(username,password);
    server.sendmail(fromaddr, toaddr, msg) # -- ' + time.ctime() )
    server.quit()

        

In [7]:
# define how to start up temperature reading
def readtemp(folderOI):
    # how ofter would you like to record temp data comes after -l flag
    # folder name for placement of .txt file should not include end "/"
    fname = folderOI + '/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_temp.txt'
    print(fname)
    syscommand = '/home/gravishlab/Documents/temperv14/temperv14 -l30 \'' + fname + '\''
    
    while(True):
        os.system(syscommand)
        curr_hour = datetime.datetime.now().hour
        if (not i.isAlive()) or (curr_hour >= hStop):
            print('Either input is dead or outside recording time -- not restarting temperv14')
            break
        
    return;



def killtemp():
    PROCNAME = "temperv14"
    for proc in psutil.process_iter():
        if proc.name() == PROCNAME:
            proc.kill()
            print('---Killed temperv14')


## Main body of program will automatically start recording sessions during specified timerange. Each session sets up parallel threads of user input, reading webcams, and recording from FLIR cams. 

In [8]:
# MAIN BODY OF PROGRAM
program_start_time = time.ctime()
print('START OF PROGRAM!', program_start_time)

# find out how many cameras actually attached
proc = subprocess.Popen("lsusb -d 1e10:4000", shell = True, stdout = subprocess.PIPE)
ncams = int(len(proc.communicate()[0].decode('UTF-8'))/60);
if len(CamNums) > ncams:
    print('number of specified cameras is not the same as those plugged in!')
    
# set up user input to close down program at any time
i=threading.Thread(target=get_input, args = (CamNums,))
i.start()
    
# start with zero sessions recorded
nSessions = 0
n_vidsanalyzed = 0

while nSessions < nDays:
    
    if (not i.isAlive()):
        killed = killallprocesses();
        killtemp()
        break
    
    curr_time = datetime.datetime.now()
    #if (curr_time.day in recording_days) and
    if (curr_time.day in recording_days) and (curr_time.hour >= hStart) and (curr_time.hour < hStop):
        
        # START UP RECORDING FOR THE DAY
        print('>>>>>>>>> Starting recording for the day\n\n')
        
        # start webcams and FLIR cameras
        n=threading.Thread(target=cams, args =(CamNums, rec_interval))
        n.start()
        continue_running = True # continue session until stop time
        
        # define folder of interest based on recording date - make folder and subfolder if doesn't exist
        folderOI = def_folderOI(curr_time, recording_days, Subs)
        
        # start temperature reading
        t=threading.Thread(target=readtemp, args =([folderOI]))
        t.start()

        # set initial timer to restart webcams and recordcam functions every ___ minutes
        start_time=time.time()
        
        # send a text message to say the program has started
        msg = 'Started recording videos! ' 
        #sendtxt(msg)
        
        while(continue_running):
            
            # Stop today's recording session?
            curr_hour = datetime.datetime.now().hour
            if curr_hour >= hStop:
                print('\nOutside of specified recording time range. Closing programs...')
                killtemp()
                killed = killallprocesses()
                continue_running = False
                nSessions = nSessions + 1
                print ('\n>>>>>>>>> Completed recording for %i session(s)!' % nSessions)
                msg = 'Completed recording session!'
                sendtxt(msg)
                break
            
#             # Play sound every ___ minutes
#             if (elapsed_time % (a_spacing*60)) < dur:
#                 p = threading.Thread(target = PlayAudio, args = (vol, dur, fr))
#                 p.start()
#                 time.sleep(dur)

            # Open air valve every ___ minutes
            curr_min = datetime.datetime.now().minute
            if (curr_hour >= air_start_time) & (curr_hour< air_stop_time) & (((curr_min-1)%a_spacing)==0) : # stop based on clock time (so can removed close trials)
#             if (elapsed_time % (a_spacing*60)) < 5: # within 5 seconds on desired spacing time
                p = threading.Thread(target = OpenValve, args = (n_repeat, spacing_sec))
                p.start()
                print('Air valve opened: ', time.ctime())
                time.sleep(60)
            
            # how long has it been since when program last started?
            elapsed_time=time.time()-start_time
            
            # close programs if input says so, otherwise restart if webcam dead or certain time has elapsed
            if (elapsed_time > restart_time)or (not i.isAlive()):# or (not n.isAlive()):  # close programs if...

                if elapsed_time > restart_time:
                    print('\nScheduled restarting of programs...')
                elif not n.isAlive():
                    print('\nA webcam has died. Restarting programs...')
                    msg = '---A webcam died <sadface>'
                    sendtxt(msg)

                # close all processes
                killed = killallprocesses();

                # restart webcams and record cam programs if haven't exited whole program
                if i.isAlive():
                    print('\nRestarting...', time.ctime())
                    n=threading.Thread(target=cams, args =(CamNums, rec_interval))
                    n.start()
                    # restart timer
                    start_time=time.time()
                else:
                    continue_running = False

            # check to see if should restart every 1 s 
            time.sleep(1) 
            
            
    # ANALYZE VIDEOS IF OVERNIGHT
    if (not curr_time.day in recording_days) or (curr_time.hour >= hStop) or (curr_time.hour < hStart-1):
        
        vids = glob.glob('/media/gravishlab/SeagateExpansionDrive/AntTrack/*162767*')
        vids = sorted(vids)
        nvids = len(vids)
        
        if nvids > 0:
            
            if not i.isAlive():
                print('User input has ended. Closing...')
                break
            
            
            if n_vidsanalyzed == 0:
                print('Analyzing ' + str(nvids) + ' videos! -- ' + time.strftime('%I:%M %p') + '\n')
                msg = 'Analyzing ' + str(nvids) + ' videos!' # ' -- ' + time.strftime('%I:%M %p')
#                 sendtxt(msg)


            sep = 50 # separation from beginning and end of video --> frames that will analyze
            buffer = 50 # how many pixels near walls to ignore (don't count ants walking on walls)

            for vname in vids:

                # look to see if ants in movie --- move to folderOI or to recycle bin
                filename = deleteormove(vname, sep, buffer, sub_orders, CamNums, Subs, recording_days) # vname, sep, buffer, dest to move
                n_vidsanalyzed = n_vidsanalyzed + 1

                # if file was moved, analyze
                if filename:
                    trackvid(filename, 500, 0.2) # filename, min object size, threshold value
                    if filename.split('.')[-1] == 'avi':
                        compressvid(filename)
                    else:
                        print('---Video already compressed')
            
            if n_vidsanalyzed > 0:
                if n_vidsanalyzed % 50 == 0:
                    msg = 'Finished analyzing ' + str(n_vidsanalyzed) + ' videos!'
                    sendtxt(msg)
                    print('\nAnalyzed ' + str(n_vidsanalyzed) + ' videos -- ' + time.strftime('%I:%M %p') + '\n')
                    
            if n_vidsanalyzed == nvids:
                msg = 'Finished analyzing ' + str(n_vidsanalyzed) + ' videos!' # ' -- ' + time.strftime('%I:%M %p')
                sendtxt(msg)
                print('\nFinished analyzing ' + str(n_vidsanalyzed) + ' videos! -- ' + time.strftime('%I:%M %p') + '\n')
            
                

print ('\nEND OF PROGRAM!')
        
        
    

START OF PROGRAM! Tue Feb 25 14:21:13 2020

Analyzing 1278 videos! -- 02:21 PM

/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061327_16276735-0000.avi  -- MOVED to Array -- 1.90 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061331_16276736-0000.avi  -- MOVED to Flat -- 3.69 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061550_16276712-0000.avi  -- MOVED to Step -- 1.71 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061550_16276718-0000.avi  -- DELETED -- 1.85 s -- 0 pix diff
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061550_16276735-0000.avi  -- MOVED to Array -- 1.64 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061558_16276736-0000.avi  -- MOVED to Flat -- 5.39 s
---Contours and tracks saved

/home/gravishlab/Documents/Python/Tracker/Tracker/Tracker.py:751: RuntimeWarning: invalid value encountered in double_scalars
  if front_pix_sum / back_pix_sum > 1.1:
/home/gravishlab/Documents/Python/Tracker/Tracker/Tracker.py:753: RuntimeWarning: invalid value encountered in double_scalars
  elif front_pix_sum / back_pix_sum < 0.9:


---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061759_16276712-0000.avi  -- MOVED to Step -- 1.72 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061759_16276718-0000.avi  -- DELETED -- 5.63 s -- 7 pix diff
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061801_16276736-0000.avi  -- DELETED -- 1.94 s -- 0 pix diff
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061853_16276735-0000.avi  -- MOVED to Array -- 1.65 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061902_16276712-0000.avi  -- MOVED to Step -- 1.66 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_061903_16276718-0000.avi  -- MOVED to C_Step -- 4.03 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTr

/home/gravishlab/.virtualenvs/ants/lib/python3.5/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_062003_16276712-0000.avi  -- MOVED to Step -- 1.65 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_062004_16276718-0000.avi  -- MOVED to C_Step -- 1.75 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_062007_16276736-0000.avi  -- MOVED to Flat -- 1.69 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_062057_16276735-0000.avi  -- MOVED to Array -- 1.86 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_062105_16276718-0000.avi  -- MOVED to C_Step -- 1.96 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_062109_16276712-0000.avi  -- MOVED to Step -- 1.86 s
---Co

/home/gravishlab/Documents/Python/Tracker/Tracker/Tracker.py:751: RuntimeWarning: divide by zero encountered in double_scalars
  if front_pix_sum / back_pix_sum > 1.1:


---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_065047_16276718-0000.avi  -- MOVED to C_Step -- 1.95 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_065055_16276736-0000.avi  -- MOVED to Flat -- 1.89 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_065102_16276712-0000.avi  -- MOVED to Step -- 1.97 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_065137_16276735-0000.avi  -- MOVED to Array -- 1.94 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_065149_16276718-0000.avi  -- MOVED to C_Step -- 2.25 s
---Contours and tracks saved
---Video file compressed
/media/gravishlab/SeagateExpansionDrive/AntTrack/20200225_065156_16276736-0000.avi  -- MOVED to Flat -- 1.91 s
---Co

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials j4sm1493876pfh.152 - gsmtp')

In [ ]:
killallprocesses()

Closing processes...
---closed webcam 0 , PID:  20176
---closed webcam 1 , PID:  20179


In [15]:
# TEST GETTING WEBCAMS TO WORK
# from ParallelCams.TriggerOneCam import *
# how many webcams need (2 point grey to every webcam)
interval = rec_interval
procs = []

nWCams = int(len(CamNums)/2)+(len(CamNums) % 2 > 0) # always rounds up
for wCamNum in range(0,2):#nWCams):
    if invert_wcams:
        relevant_cams = np.split(np.array(CamNums), 2)[::-1][wCamNum]
    else:
        relevant_cams = np.split(np.array(CamNums), 2)[wCamNum]
    #print(CamNum)
    p1 = Process(target=startwebcam, args = (wCamNum, relevant_cams, interval))
    p1.start()
    procs.append(p1)

Starting webcam 0 for cams [35 36]...
Starting webcam 1 for cams [12 18]...
ret is false


Process Process-14:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gravishlab/Documents/Python/ParallelCams/TriggerOneCam.py", line 81, in startwebcam
    if not frame.any():
AttributeError: 'list' object has no attribute 'any'


Keystroke closing webcam


In [17]:
# TESTING WEBCAM WIDTH AND HEIGHT
# from ParallelCams.TriggerOneCam import *

import cv2
print('done importing')
cap = cv2.VideoCapture(1)
fgbg = cv2.createBackgroundSubtractorMOG2(history = 120, varThreshold = 7, detectShadows=False)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
# cap.set(4, 1280)
# cap.set(5, 720)
print('set height and width')

# to set up background subtract
ret, frame = cap.read()

while(True):

    if not ret:
        print('****** ret is false ******')
        break
    
    # Display normal frame
    frame = readanddisplaywebcam(cap, 0)
#                                                                                              

#     # adjust for dark
#     max_pix = 100
#     if np.mean(frame)<70:
# #         print('dark')
#         frame[frame>max_pix] = max_pix
#         frame = (frame / max_pix * 255).astype(np.uint8)
    
    # for crappy webcams
#     ROI1=[40,260,200,280]; # cam 12
#     ROI2=[340,560,200,280]; # cam 18
    ROI1=[35,260,200,280]; # cam 35
    ROI2=[340,550,200,280]; # cam 36


    f_roi1 = frame[ ROI1[2]:ROI1[3], ROI1[0]:ROI1[1]]
    cv2.imshow('ROI1',f_roi1)
#     f_roi2 = frame[ ROI2[2]:ROI2[3], ROI2[0]:ROI2[1]]
#     cv2.imshow('ROI2',f_roi2)

#     bkgdsub, opened = transformimage(frame, fgbg)
    bkgdsub = fgbg.apply(frame)
    ret,thresholded = cv2.threshold(bkgdsub, 150, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement( cv2.MORPH_RECT, (3,3))
    opened = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)
    
    o_roi1 = opened[ ROI1[2]:ROI1[3], ROI1[0]:ROI1[1]]
    
    cv2.imshow('bkgd', bkgdsub)
    cv2.imshow('ROI bksub and opened', np.vstack([bkgdsub[ ROI1[2]:ROI1[3], ROI1[0]:ROI1[1]],
               o_roi1]))
    
#     o_cutoff = 30
#     if o_roi1.sum()/255>o_cutoff:
#         print(o_roi1.sum()/255)
        
    
#     ret,thresholded = cv2.threshold(f_roi1, 120, 255, cv2.THRESH_BINARY)
#     kernel = cv2.getStructuringElement( cv2.MORPH_RECT, (3,3))
#     opening = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)
#     cv2.imshow('Thresholded', thresholded)
#     cv2.imshow('Opened', opening)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print('Keystroke closing webcam')
        print(frame.shape)
        break
        
cap.release()
cv2.destroyAllWindows()

done importing
set height and width
Keystroke closing webcam
(480, 640, 3)


In [14]:
# # find list of all videos
# vids = glob.glob('/media/gravishlab/Seagate Expansion Drive/AntTrack/*18-0000.avi')
# print('Total Number of Videos: ',len(vids))
# vids = sorted(vids)

# # dest folder
# dfolder = '/media/gravishlab/Seagate Expansion Drive/AntTrack/AntTunnel/5mm_clear' # if running as part of trigger cams
# # dfolder = '/media/gravishlab/Seagate Expansion Drive/AntTrack/0mm_clear'
# if not os.path.exists(dfolder):
#     os.makedirs(dfolder)